In [ ]:
from datetime import datetime, timedelta
import numpy as np
import os
import pandas as pd
import zipfile
import tempfile

In [ ]:
data = pd.read_csv(r"/Users/ankitakhatri/Downloads/EQR_2019_Q4_data.csv")
data

In [ ]:
data_deleted_data = pd.DataFrame(columns=data.columns)

In [ ]:
# create new df to do all the cleaning on and convert stuff to datetime 
data_clean = data.copy()
data_clean['transaction_end_date'] = pd.to_datetime(data_clean['transaction_end_date'], format = '%Y%m%d%H%M') # convert to datetime 
data_clean['transaction_begin_date'] = pd.to_datetime(data_clean['transaction_begin_date'], format = '%Y%m%d%H%M') # convert to datetime 
data_clean.loc[data_clean['transaction_end_date'].dt.minute == 0, 'transaction_end_date'] -= pd.Timedelta(minutes = 1) # change everything to end in 59 minutes for consistency
data_clean

In [ ]:
data_clean['converted_price'] = pd.NA
data_clean['converted_rate_unit'] = pd.NA

In [ ]:
# drop any rows where price is NA 
na_rows = data_clean[data_clean['price'].isna()]
data_deleted_data = pd.concat([data_deleted_data, na_rows], ignore_index=True)
data_clean = data_clean.dropna(subset=['price']).reset_index(drop=True)
print(len(data_clean))
data_clean

In [ ]:
# price conversions to $/KW-MO for corresponding rate units, anything that is $/MW is deleted because it is frequency response 
for index, row in data.iterrows():
    price = row['price']
    rate_unit = row['rate_units']

    if rate_unit =='$/KW-MO':
        data_clean.at[index, 'converted_price'] = price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'

    elif rate_unit == '$/MW-MO':
        converted_price = price / 1000
        data_clean.at[index, 'converted_price'] = converted_price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'
    
    elif rate_unit == '$/MW-DAY':
        converted_price = (price * 365 / 12) / 1000
        data_clean.at[index, 'converted_price'] = converted_price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'

    elif rate_unit == '$/KW-DAY':
        converted_price = price * (365 / 12)
        data_clean.at[index, 'converted_price'] = converted_price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'

    elif rate_unit == '$/MW':
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        data_clean.drop(index, inplace=True)
    
    elif rate_unit == '$/KW':
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        data_clean.drop(index, inplace=True)

data_clean

In [ ]:
# drop any rows where converted price is NA 
na_rows = data_clean[data_clean['converted_price'].isna()]
data_deleted_data = pd.concat([data_deleted_data, na_rows], ignore_index=True)
data_clean = data_clean.dropna(subset=['converted_price']).reset_index(drop=True)
print(len(data_clean))
data_clean

In [ ]:
# NA value check 
index = 0
for price in data_clean['converted_price']:
    index +=1
    if pd.isna(price) == True:
        print(f"NA converted price index: {index}")

In [ ]:
# take out 0 and less
rows_to_delete = []

for index, row in data_clean.iterrows():
    if row['converted_price'] <= 0:
        rows_to_delete.append(index)

rows_to_append = data_clean.loc[rows_to_delete]
data_clean.drop(rows_to_delete, inplace=True)
data_deleted_data = pd.concat([data_deleted_data, rows_to_append], ignore_index = True)

In [ ]:
# organize delivery begin and end date columns 
data_clean['begin_date'] = data_clean['transaction_begin_date'].copy()
data_clean['end_date'] = data_clean['transaction_end_date'].copy()

data_clean['date_difference'] = data_clean['end_date'] - data_clean['begin_date']
data_clean['transaction_days'] = data_clean['date_difference'] + timedelta(minutes = 1)

# data_clean['delivery_month'] = data_clean['begin_date'].dt.month
data_clean['delivery_month'] = data_clean['begin_date'].dt.strftime('%Y-%m')

data_clean

In [ ]:
# delete any transactions that are shorter or longer than a month 
deleted_indices = []

for index, row in data_clean.iterrows():
    days = row['transaction_days'].days
    if days < 25:
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        deleted_indices.append(index)

    elif days > 31:
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        deleted_indices.append(index)

data_clean = data_clean.drop(deleted_indices)

print(len(data_clean))
data_clean

In [ ]:
print(data_clean['converted_price'].mean())
print(data_clean['converted_price'].std())
print(data_clean['converted_price'].min())
print(data_clean['converted_price'].max())
data_clean

In [ ]:
average = data_clean['converted_price'].mean()
stdev = data_clean['converted_price'].std()
print(average)
print(stdev)

In [ ]:
# remove less than 1 normal stdevs 
threshold = average - stdev
print(threshold)

indices_to_delete = []
for index, row in data_clean.iterrows():
    converted_price = row['converted_price']
    # print(f"Converted Price: {converted_price}")
    if converted_price < threshold:
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        indices_to_delete.append(index)

data_clean = data_clean.drop(indices_to_delete)

print(len(data_clean))
data_clean

In [ ]:
print(len(indices_to_delete))

In [ ]:
print(data_clean['converted_price'].min())
print(data_clean['converted_price'].max())

In [ ]:
i = 0
for price in data_clean['converted_price']:
    i +=1
    if pd.isna(price) == True:
        print(f"NA converted price index: {index}")

In [ ]:
print(len(data_clean))
data_clean

In [ ]:
data_clean['standardized_quantity'] = data_clean['standardized_quantity'].apply(
    lambda x: x / 1000 if x > 10000 else x
)

In [ ]:
data_clean['product'] = data_clean['standardized_quantity'] * data_clean['converted_price']
sumproduct = data_clean['product'].sum()
print(sumproduct)
sum_weights = data_clean['standardized_quantity'].sum()
weighted_avg = sumproduct / sum_weights
print(f"Weighted Average: {weighted_avg}")

diff_squared = (data_clean['converted_price'] - weighted_avg) ** 2
sumproduct_diff = np.sum(diff_squared * data_clean['standardized_quantity'])
weighted_std_dev = np.sqrt(sumproduct_diff / (sum_weights - 1))
print(f"Weighted Std Dev: {weighted_std_dev}")

In [ ]:
# convert trade date to datetime then extract into new column the formatting wanted
data_clean['trade_date'] = pd.to_datetime(data_clean['trade_date'], format = '%Y%m%d')
data_clean['trade_date_year_mo'] = data_clean['trade_date'].dt.strftime('%Y-%m')
data_clean

In [ ]:
date_range = pd.date_range(start= '2013-01-01', end= '2024-12-01', freq= 'MS')
trade_date_year_mo = date_range.strftime('%Y-%m')
output_df = pd.DataFrame()

for delivery_month in data_clean['delivery_month'].unique():
    df = pd.DataFrame({'trade_date_year_mo': trade_date_year_mo})

    weighted_avg_prices = []
    total_quantities = []

    for year_mo in trade_date_year_mo:
        mask = (data_clean['trade_date_year_mo'] == year_mo) & (data_clean['delivery_month'] == delivery_month)
        filter_data = data_clean[mask]

        weighted_avg_price = 0
        total_quantity = 0

        if not filter_data.empty:
            total_quantity = filter_data['standardized_quantity'].sum()
            if total_quantity > 0:
                weighted_avg_price = (filter_data['standardized_quantity'] * filter_data['converted_price']).sum() / total_quantity
            else:
                weighted_avg_price = float('nan')
        else:
            weighted_avg_price = float('nan')
            total_quantity = float('nan')

        weighted_avg_prices.append(weighted_avg_price)
        # print(weighted_avg_prices)
        total_quantities.append(total_quantity)
        # print(total_quantities)
    print(f"delivery month: {delivery_month}")
    df['delivery_year_mo'] = delivery_month
    df['weighted_avg_price'] = weighted_avg_prices
    df['total_transacted_quantity'] = total_quantities
    print(df)
    

    output_df = pd.concat([output_df, df], ignore_index= True)
    print(f"output df length: {len(output_df)}")
    output_df

In [ ]:
data_deleted_data

In [ ]:
data_clean_csv_path = r"/Users/ankitakhatri/Downloads/EQR_2019_Q4_clean.csv"
data_clean.to_csv(data_clean_csv_path, index=False)

In [ ]:
output_df_csv_path = r"/Users/ankitakhatri/Downloads/output_df_2019_Q4.csv"
output_df.to_csv(output_df_csv_path, index=False)

In [ ]:
data_deleted_data_csv_path = r"/Users/ankitakhatri/Downloads/data_deleted_data_2019_Q4.csv"
data_deleted_data.to_csv(data_deleted_data_csv_path, index=False)